## Import Libaries

In [29]:
import torch
import torch
import torch.nn.functional as F
import torchvision.transforms as transforms

In [33]:
# import local modules
from MODNet.src.models.modnet import MODNet

modnet = torch.nn.DataParallel(MODNet())



cannot find the pretrained mobilenetv2 backbone


NameError: name 'exit' is not defined

In [17]:
image = "example_1.png"

print('''"''' + "{{url_for('static', filename =" + f"'bgrm_result/{image}'" + ")}}" + '''"''' )

"{{url_for('static', filename ='bgrm_result/example_1.png')}}"


In [34]:
from natsort import natsorted, ns
from PIL import Image

import os
import sys
import cv2
import glob
import gdown
import torch
import shutil 
import numpy as np
import pandas as pd

import torch.nn.functional as F
import torchvision.transforms as transforms

## Download U2Net, MODNet and models

In [38]:
# if repository not already in directory, clone it
if not os.path.exists('U-2-Net'):
  !git clone https://github.com/xuebinqin/U-2-Net.git

# if repository not already in directory, clone it
if not os.path.exists('MODNet'):
  !git clone https://github.com/ZHKKKe/MODNet.git

# create the folder to save model
if not os.path.exists(r'./U-2-Net/saved_models/u2net/'):
    os.mkdir(r'./U-2-Net/saved_models/u2net/')

# download U-2-net model 
if not os.path.isfile(r'./U-2-Net/saved_models/u2net/u2net.pth'):
    url = 'https://drive.google.com/uc?id=1ao1ovG1Qtx4b7EoskHXmi2E9rp5CHLcZ'
    output = r'./U-2-Net/saved_models/u2net/u2net.pth'
    gdown.download(url, output, quiet=False)

# download modnet_mobilenetn2 model 
if not os.path.isfile(r'./MODNet/pretrained/mobilenetv2_human_seg.ckpt'):
    url = 'https://drive.google.com/uc?id=1gNJXQPUBBp2mbA4q1Giz5mzv3EpxR7lq'
    output = r'./MODNet/pretrained/mobilenetv2_human_seg.ckpt'
    gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1gNJXQPUBBp2mbA4q1Giz5mzv3EpxR7lq
To: c:\01 D\ML project\Remove_Image_Background\MODNet\pretrained\mobilenetv2_human_seg.ckpt
9.09MB [00:03, 2.59MB/s]


In [5]:
# Computing TRIMAP
def compute_trimap_from_alpha(name, src_dir, dst_dir, open_size=10,
                              alpha_margin=10):
    src_path = os.path.join(src_dir, name)
    # if not os.path.exists(src_path):
    #     logger.error('"%s" dose not exist', src_path)
    #     return
    print(src_path)

    dst_path = os.path.join(dst_dir, name)
    print(dst_path)
    # if os.path.exists(dst_path):
    #     logger.info('"%s" exists, Skip...', dst_path)
    #     return

    # logger.info('Trimap for "%s"', src_path)
    alpha = cv2.imread(src_path, 0)
    print(alpha)
    assert alpha.ndim == 2

    # Compute each region
    fore = ((255 - alpha_margin) < alpha)
    back = (alpha < alpha_margin)
    unknown = ~(fore + back)
    unknown = cv2.dilate(
        unknown.astype(np.uint8),
        cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (open_size, open_size))
    ).astype(np.bool)

    # Draw
    trimap = np.zeros_like(alpha)
    trimap[fore] = 255
    trimap[unknown] = 127

    cv2.imwrite(dst_path, trimap)

In [6]:
# Loading data in correct format for MODNet training  
def dataloader(file_location):
    path = file_location
    image_path = os.listdir(path + 'images')
    gt_path = os.listdir(path + 'ground_truths')
    trimap = os.listdir(path + 'trimaps')

    # short in an order
    image_path = natsorted(image_path, alg=ns.IGNORECASE)
    gt_path = natsorted(gt_path, alg=ns.IGNORECASE)
    trimap = natsorted(trimap, alg=ns.IGNORECASE)

    return zip(image_path, trimap, gt_path)

In [45]:
# getting trimap and manging storage and memory
def get_trimap(path, image):

    # main image dir should have 3 important folders images, ground_truths, trimaps
    # (one image at a time) copy target image to ./U-2-Net/test_image
    shutil.copy2(f'{path}images/{image}', './U-2-Net/test_data/test_images')

    # generate alpha_matting
    %cd ./U-2-Net/
    !python u2net_test.py

    # generate trimap 
    %cd ..
    compute_trimap_from_alpha(f"{image[:-4]}.png", "./U-2-Net/test_data/u2net_results/", f'{path}trimaps/')

    # delete every image file from test_image and u2net_results
    os.remove(f'./U-2-Net/test_data/test_images/{image}')
    
    shutil.move(f'./U-2-Net/test_data/u2net_results/{image[:-4]}.png', f'{path}alpha_matting/')


In [ ]:
# geting trimap of every image from the test and traing set
paths = ['./dataset/training_set/', './dataset/test_set/']
images = os.listdir(f'{path}/images')

# uncomment to generete trimaps (WARNING TAKES LOT OF TIME)
"""
for path in paths:
    for image in images:
        get_trimap(path, image)
"""

In [4]:
def process_image(image_path, binary=False):
  """
  Function to process image into the input format required
  for model
  """ 
  # read image
  im = Image.open(image_path)

  # define image to tensor transform
  im_transform = transforms.Compose(
      [
          transforms.ToTensor(),
          transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
      ])

  gt_transform = transforms.Compose(
      [
          transforms.ToTensor(),
          transforms.Normalize((0.5), (0.5))
      ])

  # unify image channels to 3
  if binary == False:
      
    im = np.asarray(im)
    if len(im.shape) == 2:
        im = im[:, :, None]
    if im.shape[2] == 1:
        im = np.repeat(im, 3, axis=2)
    elif im.shape[2] == 4:
        im = im[:, :, 0:3]
        
    # convert image to PyTorch tensor
    im = Image.fromarray(im)
    im = im_transform(im).cuda()

  else:
    im = im.convert('1') 
    im = np.asarray(im)

    # convert image to PyTorch tensor
    im = Image.fromarray(im)
    im = gt_transform(im).cuda()

  # add mini-batch dim
  im = im[None, :, :, :]

  # resize image for input
  im_b, im_c, im_h, im_w = im.shape

  if max(im_h, im_w) < ref_size or min(im_h, im_w) > ref_size:
      if im_w >= im_h:
          im_rh = ref_size
          im_rw = int(im_w / im_h * ref_size)
      elif im_w < im_h:
          im_rw = ref_size
          im_rh = int(im_h / im_w * ref_size)
  else:
      im_rh = im_h
      im_rw = im_w

  im_rw = im_rw - im_rw % 32
  im_rh = im_rh - im_rh % 32
  im = F.interpolate(im, size=(im_rh, im_rw), mode='area')

  return im

In [5]:
def get_image_paths(image_dir_list: list) -> pd.DataFrame:
  """
  Function to get a dataframe with the different paths for each image
  """
  # create empty list
  image_df_list = [] # will contain a dataframe for each directory

  # create dataframes with list of image paths per directory
  for image_dir in image_dir_list:
    image_path_list = glob.glob(image_dir + os.sep + '*')
    image_df = pd.DataFrame(image_path_list, columns=[image_dir])

    # create a filename column by extracting the image filename without extension
    image_df["filename"] = image_df[image_dir].str.split(os.sep).str[-1].str.split(".").str[0]

    # append dataframe to dataframe list
    image_df_list.append(image_df)
    print(image_df.shape)

  # merge dataframes on the image filename
  df = pd.merge(image_df_list[0], image_df_list[1], on="filename")
  print(df.shape)
  df = pd.merge(df, image_df_list[2], on="filename")
  print(df.shape)

  return df

In [ ]:
def dataloader(image_paths_df, batch_size):
  """
  Function to load the model with the paths of:
  * original images
  * trimaps 
  * ground truth images
  """
  
  np.array_split(df, 16)

  paths_list = []
  print(image_paths_df.columns)
  for column_name in image_paths_df.columns:
    if column_name != "filename":
      print(column_name)
      paths = image_paths_df[column_name].to_list()
      paths_list.append(paths)
    
  image_list, trimap_list, gt_path_list = paths_list[0], paths_list[1], paths_list[2]

  return zip(image_list, trimap_list, gt_path_list)